In [9]:
import matplotlib.pyplot as plt
from torch import nn
import sys
import torch
import numpy as np
from pathlib import Path

sys.path.append(r"C:\Users\Federico\Documents\GitHub\pysical_locomotion")
sys.path.append("./")

from pyrnn import RNN
from torch.utils.data import Subset
from fcutils.path import subdirs
from sklearn.model_selection import train_test_split

# from pyrnn.render import render_state_history_pca_3d
from analysis.RNN.task import (
    # make_batch,
    GoalDirectedLocomotionDataset,
    plot_predictions,
)

In [2]:
def train_val_dataset(dataset, val_split=0.25):
    train_idx, val_idx = train_test_split(
        list(range(len(dataset))), test_size=val_split
    )
    datasets = {}
    datasets["train"] = Subset(dataset, train_idx)
    datasets["test"] = Subset(dataset, val_idx)
    return datasets

# Intro
Having trained RNN on goal directed locomotion tasks with varying length of track trajectory horizon information and $\delta t$ of future velocities predictions, we compare their performance here. 

In [3]:
datasets_folder = Path(r"D:\Dropbox (UCL)\Rotation_vte\Locomotion\analysis\RNN\datasets")
results_folders = subdirs(Path(r"D:\Dropbox (UCL)\Rotation_vte\Locomotion\analysis\RNN\datasets_comparison"))
names = [f.name for f in results_folders]

In [10]:
# load all datasets & networks
datasets = {}
networks = {}
for fld in results_folders:
    task = GoalDirectedLocomotionDataset(
        max_dataset_length=3000, 
        data_folder= datasets_folder / fld.name,
        stored_scalers_path = fld,
    )

    datasets[fld.name] = torch.utils.data.DataLoader(
            train_val_dataset(task)["test"],  # get the test dataset,
            batch_size=256,
            num_workers=0,
            shuffle=True,
            worker_init_fn=lambda x: np.random.seed(),
            pin_memory=False,
            # collate_fn=pad_pack_collate,
        )
    
    
    
    networks[fld.name] = RNN.load(
        str(fld / "MIN_LOSS.pt"),
        n_units=256,
        input_size=task.n_inputs,
        output_size=task.n_outputs,
    )

Generating data... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--

[06/06/22 16:49:26] INFO     Loading model from: D:\Dropbox (UCL)\Rotation_vte\Lo ]8;id=723833;file://c:\users\federico\documents\github\pyrnn\pyrnn\_rnn.py\_rnn.py]8;;\:]8;id=357935;file://c:\users\federico\documents\github\pyrnn\pyrnn\_rnn.py#225\225]8;;\
                             comotion\analysis\RNN\datasets_comparison\5cm_pred_0            
                             .75\MIN_LOSS.pt                                                 

                    INFO     Loading dataset data at: 'D:\Dropbox (UCL)\Rotation_v ]8;id=172813;file://C:\Users\Federico\Documents\GitHub\pysical_locomotion\analysis\RNN\task.py\task.py]8;;\:]8;id=517073;file://C:\Users\Federico\Documents\GitHub\pysical_locomotion\analysis\RNN\task.py#93\93]8;;\
                             te\Locomotion\analysis\RNN\datasets\5cm_pred_1')                

[06/06/22 16:50:00] INFO     Loading normalizers                                  ]8;id=511220;file://C:\Users\Federico\Documents\GitHub\pysical_locomotion\analysis\RNN\task.py\task.py]8;;\:]8;id=168609;file://C:\Users\Federico\Documents\GitHub\pysical_locomotion\analysis\RNN\task.py#146\146]8;;\

                    INFO     ['n', 'ψ', 's', 'V', 'ω', 'k_1', 'k_2']               ]8;id=861134;file://C:\Users\Federico\Documents\GitHub\pysical_locomotion\analysis\RNN\task.py\task.py]8;;\:]8;id=325145;file://C:\Users\Federico\Documents\GitHub\pysical_locomotion\analysis\RNN\task.py#81\81]8;;\

                    INFO     ('V̇', 'ω̇')                                            ]8;id=964318;file://C:\Users\Federico\Documents\GitHub\pysical_locomotion\analysis\RNN\task.py\task.py]8;;\:]8;id=750076;file://C:\Users\Federico\Documents\GitHub\pysical_locomotion\analysis\RNN\task.py#82\82]8;;\

                    INFO     Generating dataset batches                           ]8;id=335181;file://C:\Users\Federico\Documents\GitHub\pysical_locomotion\analysis\RNN\task.py\task.py]8;;\:]8;id=566186;file://C:\Users\Federico\Documents\GitHub\pysical_locomotion\analysis\RNN\task.py#173\173]8;;\

Output()

[06/06/22 16:50:03] INFO     Loading model from: D:\Dropbox (UCL)\Rotation_vte\Lo ]8;id=921676;file://c:\users\federico\documents\github\pyrnn\pyrnn\_rnn.py\_rnn.py]8;;\:]8;id=966274;file://c:\users\federico\documents\github\pyrnn\pyrnn\_rnn.py#225\225]8;;\
                             comotion\analysis\RNN\datasets_comparison\5cm_pred_1            
                             \MIN_LOSS.pt                                                    

In [11]:
criterion = nn.MSELoss()

for name in names:
    for (X, Y) in datasets[name]:
        print(X.shape, Y.shape)

        output, h = networks[name](X)
        loss = criterion(output, Y)

        break
    break

torch.Size([256, 201, 5]) torch.Size([256, 201, 2])
